In [ ]:
# FIG 2 C
import os
os.chdir("../")
import numpy as np
import pickle
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.linalg import eigh
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, dendrogram

# Parameters
N = 400           # Number of genes
T = 300           # Total time
dt = 0.1          # Time step size
timesteps = int(T / dt)
sigma_K = 1.0     # Std dev of random K entries
lambda_base = 1.0 # Self-decay rate
D = 0.5           # Noise strength
epsilons = np.linspace(0, 0.09, 50)  # Range of coupling strengths

# Storage
top_eigvals = []
pca_variances = []
cov_matrices = []

# Precompute a single random coupling matrix
K = np.random.normal(0, sigma_K , size=(N, N))
np.fill_diagonal(K, 0)

# Loop over epsilon
for epsilon in epsilons:
    Lambda = np.diag(np.ones(N) * lambda_base)
    A = Lambda + epsilon * K

    # Simulate dynamics
    x = np.zeros((timesteps, N))
    sqrt_2Ddt = np.sqrt(2 * D * dt)
    for t in range(1, timesteps):
        dx = -A @ x[t - 1] * dt + sqrt_2Ddt * np.random.normal(size=N)
        x[t] = x[t - 1] + dx

    # Estimate steady-state covariance
    steady_state_x = x[int(timesteps / 2):]
    cov = np.cov(steady_state_x.T)
    cov_matrices.append(cov)

    # Compute eigenvalues
    eigvals = eigh(cov, eigvals_only=True)[::-1]
    top_eigvals.append(eigvals[0])
    pca_variances.append(eigvals / np.sum(eigvals))

# Convert to arrays
top_eigvals = np.array(top_eigvals)
pca_variances = np.array(pca_variances)


base_dir = "output"
out_dir = os.path.join(base_dir, "Fig2")
os.makedirs(out_dir, exist_ok=True)

plt.figure(figsize=(8, 5))
plt.plot(epsilons, pca_variances[:, 0], label='Top PCA variance')
# plt.plot(epsilons, top_eigvals/top_eigvals[-1], label='Top eigenvalue of covariance')
plt.xlabel("Coupling strength ε")
plt.ylabel("Variance explained / Eigenvalue")
plt.title("Emergence of Low-Rank Structure")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(out_dir, 'fig_pca_transition.svg'), format='svg')
plt.show()


In [ ]:
# FIG 2 D

import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eigh

# Function to simulate and compute participation ratio over ε for a given N
def simulate_participation_vs_epsilon(N, epsilons, T=300, dt=0.1, lambda_base=1.0, D=0.5):
    timesteps = int(T / dt)
    T_eff = timesteps // 2
    np.random.seed(42)
    K = np.random.normal(0, 1.0, size=(N, N))
    np.fill_diagonal(K, 0)
    participation_ratios = []
    valid_eps = []

    for epsilon in epsilons:
        Lambda = np.diag(np.ones(N) * lambda_base)
        A = Lambda + epsilon * K
        x = np.zeros((timesteps, N))
        sqrt_2Ddt = np.sqrt(2 * D * dt)
        for t in range(1, timesteps):
            dx = -A @ x[t - 1] * dt + sqrt_2Ddt * np.random.normal(size=N)
            x[t] = x[t - 1] + dx
        steady_state_x = x[int(timesteps / 2):]
        cov = np.cov(steady_state_x.T)
        if not np.isfinite(cov).all():
            continue
        eigvals = eigh(cov, eigvals_only=True)[::-1]
        total_var = np.sum(eigvals)
        total_var_sq = np.sum(eigvals**2)
        pr = (total_var**2) / total_var_sq
        participation_ratios.append(pr)
        valid_eps.append(epsilon)

    return np.array(valid_eps), np.array(participation_ratios)

# Run for different N
Ns = [400, 800, 1200]
colors = ['red', 'purple', 'blue']
epsilons = np.linspace(0.001, 0.12, 50)

# Plot rescaled participation ratio curves
plt.figure(figsize=(8, 6))

for N, color in zip(Ns, colors):
    valid_eps, pr = simulate_participation_vs_epsilon(N, epsilons)
    eps_scaled = valid_eps * np.sqrt(N)  # rescale ε axis
    plt.plot(eps_scaled, pr, label=f"N = {N}", color=color)

plt.xlabel(r"Rescaled coupling strength $\epsilon \cdot \sqrt{N}$")
plt.ylabel("Participation Ratio")
plt.title("Finite-Size Scaling Collapse of Effective Dimensionality")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(out_dir, 'fig_collapse.svg'), format='svg')
plt.show()


In [ ]:
# calculation for FIG 2 E 1 of 3

import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eigh, inv
from scipy.linalg import pinv
def compute_covariance(X0):
    return np.cov(X0, rowvar=False)
# Parameters
N = 300
T = 300
dt = 0.05
timesteps = int(T / dt)
lambda_base = 1.0
D = 0.5
epsilon = 1/(N)**.5*(1.-.05)  # subcritical but close to transition
# epsilon = 1/(N)**.5  # subcritical but close to transition
# epsilon = 1/(N)**.5*(1.+.05)  # subcritical but close to transition
np.random.seed(42)
K = np.random.normal(0, 1.0, size=(N, N))
# K = (K + K.T) / 2  # symmetrize
np.fill_diagonal(K, 0)
A = lambda_base * np.eye(N) + epsilon * K


# Compute the smallest real part of the eigenvalues of A
eigvals = np.linalg.eigvals(A)
lambda_min = np.min(np.real(eigvals))

# Estimate relaxation time
relax_time = 10 / lambda_min  # heuristic: ~5 relaxation times to reach steady state
# T = relax_time
# timesteps = int(T/dt)
print(f"Minimum eigenvalue of A (Re part): {lambda_min:.4f}")
print(f"Recommended minimum simulation time for relaxation: T ≳ {relax_time:.2f}")


# Add step input (constant forcing) to one gene
u = np.zeros(N)
perturbed_gene = 1
u[perturbed_gene] = 1.0

from numba import njit

from numba import njit
import numpy as np

# Recompile with nopython=True (enforced by @njit)
@njit(nopython=True)
def simulate_ou(A, u, with_input, timesteps, N, D, dt):
    x = np.zeros((timesteps, N))
    sqrt_2Ddt = np.sqrt(2 * D * dt)
    zero_vec = np.zeros(N)
    force = u if with_input else zero_vec
    for t in range(1, timesteps):
        noise = np.random.normal(0.0, 1.0, N)
        dx = (-A @ x[t - 1] + force) * dt + sqrt_2Ddt * noise
        x[t] = x[t - 1] + dx
    return x

# Test run with current A and u
x_perturbed = simulate_ou(A, u, True, timesteps, N, D, dt)
x_unperturbed = simulate_ou(A, u, False, timesteps, N, D, dt)



# Compute average expression over second half of time series
steady_state_x = x_unperturbed[int(timesteps / 2):]
Sigma = compute_covariance(steady_state_x)
X1 = x_perturbed[timesteps // 2:].mean(axis=0)
X0 = x_unperturbed[timesteps // 2:].mean(axis=0)
dX = X1 - X0  # observed mean shift due to step input

# Compute theoretical prediction using Σ = A⁻¹
A_inv = pinv(A)
dX_theory = A_inv @ u

 # Solve for optimal u at gene_index
sigma_column = Sigma[:, perturbed_gene]
optimal_u = np.dot(sigma_column, dX) / np.dot(sigma_column, sigma_column)
# Compute predicted response
# dX_theory = optimal_u * sigma_column

# Compare results
plt.figure(figsize=(8, 5))
plt.plot(dX, label="Simulated dX", marker='o')
plt.plot(dX_theory, label="Predicted A⁻¹u", linestyle='--')
plt.xlabel("Gene index")
plt.ylabel("Mean expression shift")
plt.title("Linear Response to Step Input (dX vs A⁻¹u)")
plt.legend()
plt.tight_layout()
plt.show()

from sklearn.metrics import r2_score

# Compute R^2 score between simulated and predicted mean shift
r2_theory = r2_score(dX, dX_theory)
# r2_sigma = r2_score(dX, dX_sigma_simple)

print(f"R² score for A⁻¹ u prediction: {r2_theory:.4f}")


# Run multiple replicates to estimate variance of dX
n_repeats = 500
dX_all = np.zeros((n_repeats, N))

for i in range(n_repeats):
    print(i)
    x_perturbed = simulate_ou(A, u, True, timesteps, N, D, dt)
    x_unperturbed = simulate_ou(A, u, False, timesteps, N, D, dt)
    X1 = x_perturbed[timesteps // 2:].mean(axis=0)
    X0 = x_unperturbed[timesteps // 2:].mean(axis=0)
    dX_all[i] = X1 - X0

# Compute mean and standard deviation of dX over replicates
dX_mean = dX_all.mean(axis=0)
dX_std = dX_all.std(axis=0)

# Plot with error bars
plt.figure(figsize=(10, 5))
plt.errorbar(np.arange(N), dX_mean, yerr=dX_std, fmt='o', label="Simulated dX ± std", capsize=2, alpha = .5)
plt.plot(dX_theory, label="Predicted A⁻¹u", linestyle='dashdot', color='orange')
plt.xlabel("Gene index")
plt.ylabel("Mean expression shift")
plt.title("Linear Response to Step Input with Error Bars")
plt.legend()
plt.tight_layout()
plt.show()


# Compute custom R²-like score accounting for uncertainty from error bars

# Define a corrected version of dX where theory is clipped to lie within error bounds
dX_lower = dX_mean - dX_std
dX_upper = dX_mean + dX_std

# Clipping predicted values to fall within the error bars
dX_clipped = np.clip(dX_theory, dX_lower, dX_upper)

# Compute squared error between clipped prediction and observed mean
ss_res = np.sum((dX_mean - dX_clipped) ** 2)

# Compute total variance of the observed mean
ss_tot = np.sum((dX_mean - np.mean(dX_mean)) ** 2)

# Custom R² score with error bar clipping
r2_corrected = 1 - ss_res / ss_tot

print(f"R² score accounting for uncertainty bounds: {r2_corrected:.4f}")


linear_data = {
    'dX_mean': dX_mean,
    'dX_std': dX_std,
    'dX_theory': dX_theory,
}

# # === Optional: save to file ===
sub_critical_out = os.path.join(out_dir, "linear_data_subcritical.pkl")
with open(sub_critical_out, "wb") as f:
    pickle.dump(linear_data, f)


In [ ]:
# calculation for FIG 2 E 2 of 3

import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eigh, inv
from scipy.linalg import pinv
def compute_covariance(X0):
    return np.cov(X0, rowvar=False)
# Parameters
N = 300
T = 300
dt = 0.05
timesteps = int(T / dt)
lambda_base = 1.0
D = 0.5
# epsilon = 1/(N)**.5*(1.-.05)  # subcritical but close to transition
epsilon = 1/(N)**.5  # critical 
# epsilon = 1/(N)**.5*(1.+.05)  # subcritical but close to transition
np.random.seed(42)
K = np.random.normal(0, 1.0, size=(N, N))
# K = (K + K.T) / 2  # symmetrize
np.fill_diagonal(K, 0)
A = lambda_base * np.eye(N) + epsilon * K


# Compute the smallest real part of the eigenvalues of A
eigvals = np.linalg.eigvals(A)
lambda_min = np.min(np.real(eigvals))

# Estimate relaxation time
relax_time = 10 / lambda_min  # heuristic: ~5 relaxation times to reach steady state
# T = relax_time
# timesteps = int(T/dt)
print(f"Minimum eigenvalue of A (Re part): {lambda_min:.4f}")
print(f"Recommended minimum simulation time for relaxation: T ≳ {relax_time:.2f}")


# Add step input (constant forcing) to one gene
u = np.zeros(N)
perturbed_gene = 1
u[perturbed_gene] = 1.0

from numba import njit

from numba import njit
import numpy as np

# Recompile with nopython=True (enforced by @njit)
@njit(nopython=True)
def simulate_ou(A, u, with_input, timesteps, N, D, dt):
    x = np.zeros((timesteps, N))
    sqrt_2Ddt = np.sqrt(2 * D * dt)
    zero_vec = np.zeros(N)
    force = u if with_input else zero_vec
    for t in range(1, timesteps):
        noise = np.random.normal(0.0, 1.0, N)
        dx = (-A @ x[t - 1] + force) * dt + sqrt_2Ddt * noise
        x[t] = x[t - 1] + dx
    return x

# Test run with current A and u
x_perturbed = simulate_ou(A, u, True, timesteps, N, D, dt)
x_unperturbed = simulate_ou(A, u, False, timesteps, N, D, dt)



# Compute average expression over second half of time series
steady_state_x = x_unperturbed[int(timesteps / 2):]
Sigma = compute_covariance(steady_state_x)
X1 = x_perturbed[timesteps // 2:].mean(axis=0)
X0 = x_unperturbed[timesteps // 2:].mean(axis=0)
dX = X1 - X0  # observed mean shift due to step input

# Compute theoretical prediction using Σ = A⁻¹
A_inv = pinv(A)
dX_theory = A_inv @ u

 # Solve for optimal u at gene_index
sigma_column = Sigma[:, perturbed_gene]
optimal_u = np.dot(sigma_column, dX) / np.dot(sigma_column, sigma_column)
# Compute predicted response
# dX_theory = optimal_u * sigma_column

# Compare results
plt.figure(figsize=(8, 5))
plt.plot(dX, label="Simulated dX", marker='o')
plt.plot(dX_theory, label="Predicted A⁻¹u", linestyle='--')
plt.xlabel("Gene index")
plt.ylabel("Mean expression shift")
plt.title("Linear Response to Step Input (dX vs A⁻¹u)")
plt.legend()
plt.tight_layout()
plt.show()

from sklearn.metrics import r2_score

# Compute R^2 score between simulated and predicted mean shift
r2_theory = r2_score(dX, dX_theory)
# r2_sigma = r2_score(dX, dX_sigma_simple)

print(f"R² score for A⁻¹ u prediction: {r2_theory:.4f}")


# Run multiple replicates to estimate variance of dX
n_repeats = 500
dX_all = np.zeros((n_repeats, N))

for i in range(n_repeats):
    print(i)
    x_perturbed = simulate_ou(A, u, True, timesteps, N, D, dt)
    x_unperturbed = simulate_ou(A, u, False, timesteps, N, D, dt)
    X1 = x_perturbed[timesteps // 2:].mean(axis=0)
    X0 = x_unperturbed[timesteps // 2:].mean(axis=0)
    dX_all[i] = X1 - X0

# Compute mean and standard deviation of dX over replicates
dX_mean = dX_all.mean(axis=0)
dX_std = dX_all.std(axis=0)

# Plot with error bars
plt.figure(figsize=(10, 5))
plt.errorbar(np.arange(N), dX_mean, yerr=dX_std, fmt='o', label="Simulated dX ± std", capsize=2, alpha = .5)
plt.plot(dX_theory, label="Predicted A⁻¹u", linestyle='dashdot', color='orange')
plt.xlabel("Gene index")
plt.ylabel("Mean expression shift")
plt.title("Linear Response to Step Input with Error Bars")
plt.legend()
plt.tight_layout()
plt.show()


# Compute custom R²-like score accounting for uncertainty from error bars

# Define a corrected version of dX where theory is clipped to lie within error bounds
dX_lower = dX_mean - dX_std
dX_upper = dX_mean + dX_std

# Clipping predicted values to fall within the error bars
dX_clipped = np.clip(dX_theory, dX_lower, dX_upper)

# Compute squared error between clipped prediction and observed mean
ss_res = np.sum((dX_mean - dX_clipped) ** 2)

# Compute total variance of the observed mean
ss_tot = np.sum((dX_mean - np.mean(dX_mean)) ** 2)

# Custom R² score with error bar clipping
r2_corrected = 1 - ss_res / ss_tot

print(f"R² score accounting for uncertainty bounds: {r2_corrected:.4f}")


linear_data = {
    'dX_mean': dX_mean,
    'dX_std': dX_std,
    'dX_theory': dX_theory,
}

# # === Optional: save to file ===
crit_out = os.path.join(out_dir, "linear_data_critical.pkl")
with open(crit_out, "wb") as f:
    pickle.dump(linear_data, f)


In [ ]:
# calculation for FIG 2 E 3 of 3

import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eigh, inv
from scipy.linalg import pinv
def compute_covariance(X0):
    return np.cov(X0, rowvar=False)
# Parameters
N = 300
T = 300
dt = 0.05
timesteps = int(T / dt)
lambda_base = 1.0
D = 0.5
# epsilon = 1/(N)**.5*(1.-.05)  # subcritical but close to transition
# epsilon = 1/(N)**.5  # subcritical but close to transition
epsilon = 1/(N)**.5*(1.+.05)  # subcritical but close to transition
np.random.seed(42)
K = np.random.normal(0, 1.0, size=(N, N))
# K = (K + K.T) / 2  # symmetrize
np.fill_diagonal(K, 0)
A = lambda_base * np.eye(N) + epsilon * K


# Compute the smallest real part of the eigenvalues of A
eigvals = np.linalg.eigvals(A)
lambda_min = np.min(np.real(eigvals))

# Estimate relaxation time
relax_time = 10 / lambda_min  # heuristic: ~5 relaxation times to reach steady state
# T = relax_time
# timesteps = int(T/dt)
print(f"Minimum eigenvalue of A (Re part): {lambda_min:.4f}")
print(f"Recommended minimum simulation time for relaxation: T ≳ {relax_time:.2f}")


# Add step input (constant forcing) to one gene
u = np.zeros(N)
perturbed_gene = 1
u[perturbed_gene] = 1.0

from numba import njit

from numba import njit
import numpy as np

# Recompile with nopython=True (enforced by @njit)
@njit(nopython=True)
def simulate_ou(A, u, with_input, timesteps, N, D, dt):
    x = np.zeros((timesteps, N))
    sqrt_2Ddt = np.sqrt(2 * D * dt)
    zero_vec = np.zeros(N)
    force = u if with_input else zero_vec
    for t in range(1, timesteps):
        noise = np.random.normal(0.0, 1.0, N)
        dx = (-A @ x[t - 1] + force) * dt + sqrt_2Ddt * noise
        x[t] = x[t - 1] + dx
    return x

# Test run with current A and u
x_perturbed = simulate_ou(A, u, True, timesteps, N, D, dt)
x_unperturbed = simulate_ou(A, u, False, timesteps, N, D, dt)



# Compute average expression over second half of time series
steady_state_x = x_unperturbed[int(timesteps / 2):]
Sigma = compute_covariance(steady_state_x)
X1 = x_perturbed[timesteps // 2:].mean(axis=0)
X0 = x_unperturbed[timesteps // 2:].mean(axis=0)
dX = X1 - X0  # observed mean shift due to step input

# Compute theoretical prediction using Σ = A⁻¹
A_inv = pinv(A)
dX_theory = A_inv @ u

 # Solve for optimal u at gene_index
sigma_column = Sigma[:, perturbed_gene]
optimal_u = np.dot(sigma_column, dX) / np.dot(sigma_column, sigma_column)
# Compute predicted response
# dX_theory = optimal_u * sigma_column

# Compare results
plt.figure(figsize=(8, 5))
plt.plot(dX, label="Simulated dX", marker='o')
plt.plot(dX_theory, label="Predicted A⁻¹u", linestyle='--')
plt.xlabel("Gene index")
plt.ylabel("Mean expression shift")
plt.title("Linear Response to Step Input (dX vs A⁻¹u)")
plt.legend()
plt.tight_layout()
plt.show()

from sklearn.metrics import r2_score

# Compute R^2 score between simulated and predicted mean shift
r2_theory = r2_score(dX, dX_theory)
# r2_sigma = r2_score(dX, dX_sigma_simple)

print(f"R² score for A⁻¹ u prediction: {r2_theory:.4f}")


# Run multiple replicates to estimate variance of dX
n_repeats = 500
dX_all = np.zeros((n_repeats, N))

for i in range(n_repeats):
    print(i)
    x_perturbed = simulate_ou(A, u, True, timesteps, N, D, dt)
    x_unperturbed = simulate_ou(A, u, False, timesteps, N, D, dt)
    X1 = x_perturbed[timesteps // 2:].mean(axis=0)
    X0 = x_unperturbed[timesteps // 2:].mean(axis=0)
    dX_all[i] = X1 - X0

# Compute mean and standard deviation of dX over replicates
dX_mean = dX_all.mean(axis=0)
dX_std = dX_all.std(axis=0)

# Plot with error bars
plt.figure(figsize=(10, 5))
plt.errorbar(np.arange(N), dX_mean, yerr=dX_std, fmt='o', label="Simulated dX ± std", capsize=2, alpha = .5)
plt.plot(dX_theory, label="Predicted A⁻¹u", linestyle='dashdot', color='orange')
plt.xlabel("Gene index")
plt.ylabel("Mean expression shift")
plt.title("Linear Response to Step Input with Error Bars")
plt.legend()
plt.tight_layout()
plt.show()


# Compute custom R²-like score accounting for uncertainty from error bars

# Define a corrected version of dX where theory is clipped to lie within error bounds
dX_lower = dX_mean - dX_std
dX_upper = dX_mean + dX_std

# Clipping predicted values to fall within the error bars
dX_clipped = np.clip(dX_theory, dX_lower, dX_upper)

# Compute squared error between clipped prediction and observed mean
ss_res = np.sum((dX_mean - dX_clipped) ** 2)

# Compute total variance of the observed mean
ss_tot = np.sum((dX_mean - np.mean(dX_mean)) ** 2)

# Custom R² score with error bar clipping
r2_corrected = 1 - ss_res / ss_tot

print(f"R² score accounting for uncertainty bounds: {r2_corrected:.4f}")


linear_data = {
    'dX_mean': dX_mean,
    'dX_std': dX_std,
    'dX_theory': dX_theory,
}

# # === Optional: save to file ===
supercrit_out = os.path.join(out_dir, "linear_data_supercritical.pkl")
with open(supercrit_out, "wb") as f:
    pickle.dump(linear_data, f)



In [ ]:
# FIG 2 E


import numpy as np
import matplotlib.pyplot as plt
import pickle

# === File names and labels ===
file_labels = [
    (sub_critical_out, "Subcritical"),
    (crit_out, "Critical"),
    (supercrit_out, "Supercritical"),
]

# === Store results ===
rel_errors = []
rel_errors_std = []
labels = []

# === Process each dataset ===
for filename, label in file_labels:
    with open(filename, "rb") as f:
        data = pickle.load(f)

    dX_mean = data["dX_mean"]
    dX_std = data["dX_std"]
    dX_theory = data["dX_theory"]

    # Compute relative error
    numerator = np.linalg.norm(dX_theory - dX_mean)
    denominator = np.linalg.norm(dX_theory)
    rel_error = numerator / denominator

    # Error propagation: std of ||dX_theory - dX_mean||
    propagated_std = np.sqrt(np.sum(dX_std ** 2))
    rel_error_std = propagated_std / denominator

    # Store results
    rel_errors.append(rel_error)
    rel_errors_std.append(rel_error_std)
    labels.append(label)

# === Plot ===
plt.figure(figsize=(6, 4))
x = np.arange(len(labels))
plt.errorbar(x, rel_errors, yerr=rel_errors_std, fmt='o', capsize=5, color='black')
plt.xticks(x, labels)
plt.ylabel("Relative Linear Response Error\n$\\|dX_\\mathrm{theory} - dX_\\mathrm{mean}\\| \\, / \\, \\|dX_\\mathrm{theory}\\|$")
plt.title("Relative Error Across Regimes")
plt.tight_layout()
plt.yscale('log')
plt_out = os.path.join(out_dir, "relative_error_linear_response_across_regimes.svg")
plt.savefig(plt_out, bbox_inches='tight')
plt.show()



import numpy as np
import pickle
from scipy.stats import wilcoxon


# === Load and build error vectors ===
error_dict = {}
for filename, label in file_labels:
    with open(filename, "rb") as f:
        data = pickle.load(f)

    dX_mean   = data["dX_mean"]
    dX_std    = data["dX_std"]    # not needed for the test itself
    dX_theory = data["dX_theory"]

    # elementwise relative error: |theory - mean| / |theory|
    # add a tiny eps to denominator to avoid div-by-zero
    eps = 1e-10
    err_vec = np.abs(dX_theory - dX_mean) / (np.abs(dX_theory) + eps)
    error_dict[label] = err_vec

# === One-sided Wilcoxon tests ===
sub = error_dict["Subcritical"]
crit = error_dict["Critical"]
sup = error_dict["Supercritical"]

print("Wilcoxon signed-rank tests (one-sided):\n")

# Test Subcritical < Critical
stat_sc, p_sc = wilcoxon(sub, crit, alternative="less")
print(f"Subcritical < Critical: stat = {stat_sc:.4f}, p = {p_sc:.4g}")

# Test Critical < Supercritical
stat_cs, p_cs = wilcoxon(crit, sup, alternative="less")
print(f"Critical   < Supercritical: stat = {stat_cs:.4f}, p = {p_cs:.4g}")

# (Optional) Test Subcritical < Supercritical
stat_ss, p_ss = wilcoxon(sub, sup, alternative="less")
print(f"Subcritical < Supercritical: stat = {stat_ss:.4f}, p = {p_ss:.4g}")


In [ ]:
# FIG 2 H
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import inv

# --------------------------
# Parameters
# --------------------------
N = 10           # Number of genes
K = 10.0         # Hill coefficient
gamma = 1.0      # Decay rate
G_mean = 1.0
G_std = 0.1
x_init = np.ones(N) * K
magnitudes = np.logspace(-2, 2, 60)
hill_exponents = [0.,1.0, 2.0, 4.0, 6.0, 8.]
hill_error_curves = {}

# --------------------------
# Core Functions
# --------------------------
def H(x, n):
    return x**n / (K**n + x**n)

def dH(x, n):
    return (n * K**n * x**(n - 1)) / (K**n + x**n)**2

def f(x):
    return gamma * x

def find_steady_state(G, n, x_init, tol=1e-14, max_iter=2000):
    x = x_init.copy()
    for _ in range(max_iter):
        dx = -f(x) + G @ H(x, n)
        if np.linalg.norm(dx) < tol:
            break
        x += 0.01 * dx
    return x

def simulate_perturbed_steady_state(G, n, x_init, u, tol=1e-14, max_iter=2000):
    x = x_init.copy()
    for _ in range(max_iter):
        dx = -f(x) + G @ H(x, n) + u
        if np.linalg.norm(dx) < tol:
            break
        x += 0.01 * dx
    return x

# --------------------------
# Hill Exponent Sweep
# --------------------------
for n_val in hill_exponents:
    G = np.abs(np.random.normal(loc=G_mean, scale=G_std, size=(N, N)))
    x0 = find_steady_state(G, n_val, x_init)
    H_prime_diag = np.diag(dH(x0, n_val))
    D = np.eye(N) * gamma
    J = -D + G @ H_prime_diag

    errors = []
    for mag in magnitudes:
        u = np.zeros(N)
        u[1] = mag
        # u[3] = mag
        delta_x_lin = -inv(J) @ u
        x_pert = simulate_perturbed_steady_state(G, n_val, x_init, u)
        delta_x_emp = x_pert - x0
        error = np.linalg.norm(delta_x_emp - delta_x_lin) / np.linalg.norm(delta_x_emp)
        errors.append(error)

    hill_error_curves[f"n = {n_val}"] = errors

# --------------------------
# Plotting
# --------------------------
plt.figure(figsize=(10, 6))
for label, errors in hill_error_curves.items():
    plt.plot(magnitudes, errors, marker='o', label=label)

plt.xscale('log')
plt.yscale('log')
plt.xlabel('Perturbation magnitude', fontsize=20)
plt.ylabel('Relative error of linear response', fontsize=20)
# plt.title('Effect of Hill exponent on linear response breakdown', fontsize=20)
plt.legend(fontsize=14)
# plt.grid(True)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout()
plt_out = os.path.join(out_dir, 'hill_net_n.svg')
plt.savefig(plt_out, format='svg')
plt.show()


In [ ]:
# FIG S1 A

import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import inv

# --------------------------
# Parameters
# --------------------------
N = 10           # Number of genes
K = 10.0         # Hill parameter
gamma = 1.0      # Decay rate
G_mean = 1.0
G_std = 0.1
x_init = np.ones(N) * K
magnitudes = np.logspace(-2, 2, 60)  # Per-gene perturbation magnitudes
hill_exponents = [0., 1.0, 2.0, 4.0]
max_k = 5                            # Max number of perturbed genes
n_trials = 20                        # Number of trials per setting

# --------------------------
# Core Functions
# --------------------------
def H(x, n):
    return x**n / (K**n + x**n)

def dH(x, n):
    return (n * K**n * x**(n - 1)) / (K**n + x**n)**2

def f(x):
    return gamma * x

def find_steady_state(G, n, x_init, tol=1e-12, max_iter=2000):
    x = x_init.copy()
    for _ in range(max_iter):
        dx = -f(x) + G @ H(x, n)
        if np.linalg.norm(dx) < tol:
            break
        x += 0.01 * dx
    return x

def simulate_perturbed_steady_state(G, n, x_init, u, tol=1e-12, max_iter=2000):
    x = x_init.copy()
    for _ in range(max_iter):
        dx = -f(x) + G @ H(x, n) + u
        if np.linalg.norm(dx) < tol:
            break
        x += 0.01 * dx
    return x

def generate_equal_magnitude_u(N, k, mag):
    """Return a vector u with exactly k entries equal to +mag, others zero."""
    u = np.zeros(N)
    indices = np.random.choice(N, k, replace=False)
    u[indices] = mag
    return u

# --------------------------
# Main Loop: Hill × k × magnitude
# --------------------------
all_errors = {}  # {(n_val, k): [list of errors vs mag]}

for n_val in hill_exponents:
    G = np.abs(np.random.normal(loc=G_mean, scale=G_std, size=(N, N)))
    x0 = find_steady_state(G, n_val, x_init)
    H_prime_diag = np.diag(dH(x0, n_val))
    D = np.eye(N) * gamma
    J = -D + G @ H_prime_diag
    J_inv = inv(J)

    for k in range(1, max_k + 1):
        curve = []
        for mag in magnitudes:
            rel_errors = []
            for _ in range(n_trials):
                u = generate_equal_magnitude_u(N, k, mag)
                dx_lin = -J_inv @ u
                x_pert = simulate_perturbed_steady_state(G, n_val, x0, u)
                dx_true = x_pert - x0
                err = np.linalg.norm(dx_true - dx_lin) / np.linalg.norm(dx_true)
                rel_errors.append(err)
            curve.append(np.mean(rel_errors))
        all_errors[(n_val, k)] = curve

# --------------------------
# Plotting
# --------------------------
for n_val in hill_exponents:
    plt.figure(figsize=(8, 6))
    for k in range(1, max_k + 1):
        label = f"{k} perturbed genes"
        plt.plot(magnitudes, all_errors[(n_val, k)], label=label, marker='o')
    plt.xscale('log')
    plt.yscale('log')
    plt.xlabel('Per-gene perturbation magnitude', fontsize=16)
    plt.ylabel('Relative linear response error', fontsize=16)
    plt.title(f"Hill exponent n = {n_val}", fontsize=16)
    plt.legend(fontsize=12)
    plt.tight_layout()
    plt_out = os.path.join(out_dir, f"hill_n_{n_val}_equal_mag_per_gene.svg")
    plt.savefig(plt_out, format='svg')
    plt.close()


In [ ]:
# FIG 2 G

# We'll fix several perturbation magnitudes and compute error as a function of G
n = 4.0

def find_steady_state(G, n, x_init, tol=1e-14, max_iter=2000):
    x = x_init.copy()
    for _ in range(max_iter):
        dx = -f(x) + G @ H(x, n)
        if np.linalg.norm(dx) < tol:
            break
        x += 0.01 * dx
    return x

def simulate_perturbed_steady_state(G, n, x_init, u, tol=1e-14, max_iter=2000):
    x = x_init.copy()
    for _ in range(max_iter):
        dx = -f(x) + G @ H(x, n) + u
        if np.linalg.norm(dx) < tol:
            break
        x += 0.01 * dx
    return x


    
selected_mags = [0.1, 0.3, 1.0]
G_vals = np.linspace(0.5, 3.5, 300)
error_vs_G = {mag: [] for mag in selected_mags}
def f(x):
    return gamma * x
for G_val in G_vals:
    G = np.abs(np.random.normal(loc=G_val, scale=G_std, size=(N, N)))
    x0 = find_steady_state(G, n, x_init)
    H_prime_diag = np.diag(dH(x0, n))
    D = np.eye(N) * gamma
    J = -D + G @ H_prime_diag

    for mag in selected_mags:
        u = np.zeros(N)
        u[1] = mag
        # u[3] = mag
        delta_x_lin = -inv(J) @ u
        x_pert = simulate_perturbed_steady_state(G, n, x_init, u)
        delta_x_emp = x_pert - x0
        error = np.linalg.norm(delta_x_emp - delta_x_lin) / np.linalg.norm(delta_x_emp)
        error_vs_G[mag].append(error)

# --------------------------
# Plotting
# --------------------------
plt.figure(figsize=(10, 6))
for mag in selected_mags:
    plt.plot(G_vals/2., error_vs_G[mag], marker='o', label=f"|u| = {mag}")

plt.xlabel('Interaction strength G_eff', fontsize=20)
plt.ylabel('Relative error of linear response', fontsize=20)
# plt.title('Error vs G at fixed perturbation magnitudes', fontsize=20)
plt.legend(fontsize=14)
plt.yscale('log')
# plt.grid(True)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout()
plt_out = os.path.join(out_dir, 'Fig2G.svg')
plt.savefig(plt_out, format='svg')
plt.show()


In [ ]:
 # FIG S1 B

# Redefine everything cleanly with K as an argument for modularity
# Sweep over K values for fixed G, n, and gamma
K_vals = np.linspace(.1, 25.0, 300)
selected_mags = [0.1, 0.3, 1.0]
error_vs_K = {mag: [] for mag in selected_mags}

# Fixed system parameters
G_val = 1.
n = 4.0
gamma = 1.0
def f(x):
    return gamma * x
def H(x, K_val):
    return x**n / (K_val**n + x**n)

def dH(x, K_val):
    return (n * K_val**n * x**(n - 1)) / (K_val**n + x**n)**2

def find_steady_state(G, K_val, x_init, tol=1e-12, max_iter=2000):
    x = x_init.copy()
    for _ in range(max_iter):
        dx = -f(x) + G @ H(x, K_val)
        if np.linalg.norm(dx) < tol:
            break
        x += 0.1 * dx
    return x

def simulate_perturbed_steady_state(G, K_val, x_init, u, tol=1e-12, max_iter=2000):
    x = x_init.copy()
    for _ in range(max_iter):
        dx = -f(x) + G @ H(x, K_val) + u
        if np.linalg.norm(dx) < tol:
            break
        x += 0.1 * dx
    return x

# Now rerun the K sweep
error_vs_K = {mag: [] for mag in selected_mags}

for K_test in K_vals:
    G = np.abs(np.random.normal(loc=G_val, scale=G_std, size=(N, N)))
    x0 = find_steady_state(G, K_test, x_init)
    H_prime_diag = np.diag(dH(x0, K_test))
    D = np.eye(N) * gamma
    J = -D + G @ H_prime_diag

    for mag in selected_mags:
        u = np.zeros(N)
        u[1] = mag
        # u[3] = mag
        delta_x_lin = -inv(J) @ u
        x_pert = simulate_perturbed_steady_state(G, K_test, x_init, u)
        delta_x_emp = x_pert - x0
        error = np.linalg.norm(delta_x_emp - delta_x_lin) / np.linalg.norm(delta_x_emp)
        error_vs_K[mag].append(error)

# --------------------------
# Plotting
# --------------------------
plt.figure(figsize=(10, 6))
for mag in selected_mags:
    plt.plot(K_vals, error_vs_K[mag], marker='o', label=f"|u| = {mag}")

plt.xlabel('K (Hill saturation parameter)', fontsize=20)
plt.ylabel('Relative error of linear response', fontsize=20)
# plt.title('Effect of K on linear response accuracy (fixed G, n)', fontsize=20)
plt.legend(fontsize=14)
plt.grid(True)
plt.yscale('log')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout()
plt_out = os.path.join(out_dir, 'FigS1B.svg')
plt.savefig(plt_out, format='svg')
plt.show()


In [ ]:
 # FIG S1 C

import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import inv

# Parameters
N = 10
K = 10.0
n = 4.0
G_mean = 1.0
G_std = 0.1
x_init = np.ones(N) * K
magnitudes = [0.1, 0.3, 1.0]
gamma_vals = np.linspace(0.01, 3.0, 300)
error_vs_gamma = {mag: [] for mag in magnitudes}

# Core functions with fixed n and K
def H(x):
    return x**n / (K**n + x**n)

def dH(x):
    return (n * K**n * x**(n - 1)) / (K**n + x**n)**2

def f(x, gamma):
    return gamma * x

def find_steady_state(G, gamma, x_init, tol=1e-12, max_iter=2000):
    x = x_init.copy()
    for _ in range(max_iter):
        dx = -f(x, gamma) + G @ H(x)
        if np.linalg.norm(dx) < tol:
            break
        x += 0.1 * dx
    return x

def simulate_perturbed_steady_state(G, gamma, x_init, u, tol=1e-12, max_iter=2000):
    x = x_init.copy()
    for _ in range(max_iter):
        dx = -f(x, gamma) + G @ H(x) + u
        if np.linalg.norm(dx) < tol:
            break
        x += 0.1 * dx
    return x

# Run the sweep
for gamma in gamma_vals:
    G = np.abs(np.random.normal(loc=G_mean, scale=G_std, size=(N, N)))
    x0 = find_steady_state(G, gamma, x_init)
    H_prime_diag = np.diag(dH(x0))
    D = np.eye(N) * gamma
    J = -D + G @ H_prime_diag

    for mag in magnitudes:
        u = np.zeros(N)
        u[1] = mag
        u[3] = mag
        delta_x_lin = -inv(J) @ u
        x_pert = simulate_perturbed_steady_state(G, gamma, x_init, u)
        delta_x_emp = x_pert - x0
        error = np.linalg.norm(delta_x_emp - delta_x_lin) / np.linalg.norm(delta_x_emp)
        error_vs_gamma[mag].append(error)

# Plotting
plt.figure(figsize=(10, 6))
for mag in magnitudes:
    plt.plot(gamma_vals, error_vs_gamma[mag], marker='o', label=f"mag = {mag}")

plt.xlabel('Decay rate γ', fontsize=20)
plt.ylabel('Relative error of linear response', fontsize=20)
plt.title('Effect of decay rate on linear response accuracy', fontsize=20)
plt.legend(fontsize=14)
plt.grid(True)
plt.yscale('log')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout()
plt_out = os.path.join(out_dir, 'FigS1C.svg')
plt.savefig(plt_out, format='svg')
plt.show()


In [ ]:
# FIG 2 J, K, L

import numpy as np
import matplotlib.pyplot as plt
from numba import njit
import matplotlib.gridspec as gridspec

# ---------------------------
# Parameters
# ---------------------------
N_genes = 20
T = 2*10**6
dt = 0.1
time = np.arange(T) * dt

beta = np.ones(N_genes) * 50.0
gamma = np.ones(N_genes) * 4.0
K = 25.0
n = 2.0
noise_strength = .5

# ---------------------------
# Team structure: A vs B
# ---------------------------
group_A = np.arange(0, N_genes // 2)
group_B = np.arange(N_genes // 2, N_genes)

B_on = np.zeros((N_genes, N_genes))
B_off = np.zeros((N_genes, N_genes))
randscale = .45
# Within-team activation
for i in group_A:
    for j in group_A:
        if i != j:
            B_on[i, j] = 0.3*(1+randscale*np.random.randn())
for i in group_B:
    for j in group_B:
        if i != j:
            B_on[i, j] = 0.25*(1+randscale*np.random.randn())

# Between-team inhibition
for i in group_A:
    for j in group_B:
        B_off[i, j] = 0.25*(1+randscale*np.random.randn())
        B_off[j, i] = 0.25*(1+randscale*np.random.randn())


# ---------------------------
# Sparsify B_on and B_off
# ---------------------------
sparsity_level = 0.70  # Set between 0 (dense) and 1 (very sparse)
rng = np.random.default_rng(seed=42)  # For reproducibility

# Randomly set a proportion of non-diagonal entries to zero
def sparsify_matrix(B, sparsity):
    mask = rng.random(B.shape) > sparsity
    np.fill_diagonal(mask, True)  # Keep diagonals if needed (optional)
    return B * mask

B_on = sparsify_matrix(B_on, sparsity_level)
B_off = sparsify_matrix(B_off, sparsity_level)


# ---------------------------
# Numba-accelerated functions
# ---------------------------
@njit
def hill_rate(x, K, n, k_max=1.0):
    return k_max * x**n / (K**n + x**n + 1e-8)

@njit
def simulate_expression(T, dt, beta, gamma, K, n, noise_strength, B_on, B_off, T_burnin):
    N = len(beta)
    T_total = T + T_burnin  # total steps = burn-in + recording
    x = np.zeros((T, N))    # only record after burn-in
    s = np.random.randint(0, 2, N)
    remaining_time = np.random.exponential(1.0, size=N)
    sqrt_dt = np.sqrt(dt)

    x_state = np.zeros(N)  # current expression state

    for t in range(T_total):
        input_on = B_on @ x_state
        input_off = B_off @ x_state

        k_on = hill_rate(input_on, K, n)
        k_off = hill_rate(input_off, K, n)

        for i in range(N):
            k_on[i] = min(max(k_on[i], 1e-3), 10.0)
            k_off[i] = min(max(k_off[i], 1e-3), 10.0)

        remaining_time -= dt
        for i in range(N):
            if remaining_time[i] <= 0:
                if s[i] == 0:
                    s[i] = 1
                    remaining_time[i] = np.random.exponential(1.0 / k_off[i])
                else:
                    s[i] = 0
                    remaining_time[i] = np.random.exponential(1.0 / k_on[i])

        dx = beta * s - gamma * x_state
        noise = np.random.normal(0, noise_strength, N) * sqrt_dt
        x_state = np.maximum(0.0, x_state + dt * dx + noise)

        if t >= T_burnin:
            x[t - T_burnin] = x_state

    return x
T_burnin = 0  # e.g. discard first 200,000 steps
# T = 10**6
x_control = simulate_expression(T, dt=dt, beta=beta, gamma=gamma,
                                K=K, n=n, noise_strength=noise_strength,
                                B_on=B_on, B_off=B_off,
                                T_burnin=T_burnin)



import matplotlib.pyplot as plt
import numpy as np

# Assuming x is your simulation output of shape (T, N_genes)
# Define team indices
group_A = np.arange(0, N_genes // 2)
group_B = np.arange(N_genes // 2, N_genes)

# Compute average expression over time for each team
avg_A = np.mean(x_control[:, group_A], axis=1)
avg_B = np.mean(x_control[:, group_B], axis=1)

# Plot
plt.figure(figsize=(10, 5))
plt.plot(avg_A, label='Team A (Genes 0–9)', color='blue', alpha = .4)
plt.plot(avg_B, label='Team B (Genes 10–19)', color='red', alpha = .4)
plt.xlabel("Time Step")
plt.ylabel("Average Gene Expression")
# plt.title("Average Expression Over Time: Team A vs Team B")
plt.legend()
plt.tight_layout()
plt_out = os.path.join(out_dir, 'teams_net_traj.svg')
plt.savefig(plt_out, format='svg')
plt.show()



from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# Style
sns.set(style="whitegrid", context="talk", palette="deep")

# ---- Step 1: Calculate Control Mean and Covariance
mu_control = np.mean(x_control, axis=0)
Sigma = np.cov(x_control.T)

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(Sigma, annot=False, cmap='coolwarm', square=True,
            cbar_kws={"label": "Covariance"}, xticklabels=1, yticklabels=1)

plt.title("Covariance Matrix Σ of Control Expression")
plt.xlabel("Gene Index")
plt.ylabel("Gene Index")
plt.tight_layout()
plt_out = os.path.join(out_dir, 'teams_net_corr.svg')
plt.savefig(plt_out, format='svg')
plt.show()

# ---- Step 2: Knockdown Gene 0 by a range of fractions
fractions = np.linspace(0.1, 1., 10)
fractions = [1.]
gene_idx = 0
X_deltas = []
X_preds = []

for frac in fractions:
    beta_pert = beta.copy()
    beta_pert[gene_idx] = beta[gene_idx] * (1 - frac)
    x_perturbed = simulate_expression(T, dt=dt, beta=beta_pert, gamma=gamma,
                                K=K, n=n, noise_strength=noise_strength,
                                B_on=B_on, B_off=B_off,
                                T_burnin=T_burnin)
    mu_pert = np.mean(x_perturbed, axis=0)

    delta_X = mu_pert - mu_control
    # u = np.zeros(N_genes)
    # Solve for optimal u at gene_index
    sigma_column = Sigma[:, gene_idx]
    u = np.dot(sigma_column, delta_X) / np.dot(sigma_column, sigma_column)
    # u[gene_idx] = -beta[gene_idx] * frac

    pred = u * sigma_column

    X_deltas.append(delta_X)
    X_preds.append(pred)

X_deltas = np.array(X_deltas)
X_preds = np.array(X_preds)



for i, frac in enumerate(fractions):
    fig, ax = plt.subplots(figsize=(6, 6))
    sns.scatterplot(x=X_preds[i], y=X_deltas[i], s=80, ax=ax)

    min_val = min(X_preds[i].min(), X_deltas[i].min())
    max_val = max(X_preds[i].max(), X_deltas[i].max())
    ax.plot([min_val, max_val], [min_val, max_val], 'k--', label='Ideal: y = x')

    r2 = r2_score(X_deltas[i], X_preds[i])
    cos_sim = np.dot(X_deltas[i], X_preds[i]) / (np.linalg.norm(X_deltas[i]) * np.linalg.norm(X_preds[i]))
    ax.set_title(f"{int(frac*100)}% Knockdown — R²: {r2:.2f}, Cos: {cos_sim:.2f}")
    ax.set_xlabel("Predicted Δx (Σu)")
    ax.set_ylabel("Empirical Δx")
    ax.legend()
    plt.tight_layout()
    plt_out = os.path.join(out_dir, 'teams_net_R2.svg')
    plt.savefig(plt_out, format='svg')
    plt.show()



